In [ ]:
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [ ]:
# TODO
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad, 
# 其為 Extractive QA 類摘要模型

from transformers import pipeline

利用 Hugging Face 的 text-generation model 與 Sentence Transformers embedding model 實作 QA 檢索聊天機器人。</p>
基於提供的資料集，使用 Embedding Cosine Similarity 檢索參考資料，再透過 LLM 生成答案。</p>

1. Baseline
   - 將 demo 中的資料，替換成我們提供 or 自己的資料集
   - 能夠檢索相似資料
   - 基於檢索的資料進行回答
2. Advanced（Optional）
   - Embedding 怎麼儲存？每次都要重新計算嗎？
   - 該如何處理太久以前的歷史資料？
   - 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [ ]:
from pprint import pprint

from torch import cuda, bfloat16

from transformers import pipeline
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

from sentence_transformers import SentenceTransformer


embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的可以參考 Hugging face 官方文件～</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`。

In [ ]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

In [ ]:
chatbot = pipeline(
    "text-generation", 
    model=hf_model, 
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

Demo

In [ ]:
qa_data = [
    """1. 入營注意事項：役男入營前無須先行理髮，待入營後統一安排理髮，再向役男收取費用
    2. 役男可攜帶手機(不得為大陸廠牌，且不提供充電)，部隊會集中保管，於每日夜間以定時定點方式使用。
    3. 營區不提供充電，建議攜帶拋棄式(手動)刮鬍刀；不可攜帶噴霧式液體。
    4. 役男可配戴隱形眼鏡，但因基礎訓練期間生活緊湊，而隱形眼鏡需相當時間消毒清洗，建議配戴鏡片眼鏡為宜，並可多備一副眼鏡，以供替換。
    """,
    
    """報到入營時應該文件：
    1. 徵集令。
    2. 役男本人之國民身分證正本。
    3. 私章。
    4. 健保ＩＣ卡。
    5. 郵局存摺正面影本。
    6. 替代役役男輔導需求調查表。
    7. 個人特殊醫療用品。
    """,
    
    """
    替代役訓練天數
    基礎訓練：21天(含撥交日)。(自253梯次起修正)
    專業訓練：以各分發需用機關不同而有個不同期間的專業訓練。
    """,
    
    """
    折抵役期規定
    請備妥相關證明文件，如高中（職）以上各級學校經軍訓主管驗證加註折抵役期日數之成績單、大專集訓結訓證書、
    驗退（停役）證明書或軍事學校退學（開除）證明書等正本（驗證後退還）。82年次以前出生者，
    合計不得逾30日，83-93年次以後出生者，合計不得逾15日。已受軍事入伍訓練者，請於收到徵集令時，向戶籍地區公所提出免受基礎訓練申請。
    """,
    
    """
    要當兵時，健保要辦理轉出轉入嗎?
    即將入營服常備兵役之役男需持徵集令向全民健保加保單位辦理轉出，轉出日期填報「入營當月份」，轉入單位免填，後續將由國軍單位接續辦理後續轉入程序。
    """,
    
    """當兵期間義務役薪水的入帳戶一定要郵局嗎?
    依現行規定目前有郵局、台新、土地銀行跟合作金庫這四家金融機構可使用。"""
]

In [ ]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str]):
    most_related_sentence = None
    max_similarity = 0

    for sentence in source:
        sim = calculate_cosine_similarity(
        embedding_model.encode(query),
        embedding_model.encode(sentence)
        )
        
        if sim > max_similarity:
            most_related_sentence = sentence
            max_similarity = sim
            
    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
user_query = input(">>>")
conversation = Conversation()

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)
    
    user_query = input(">>>")
    